# Naive time adaptive method

In [1]:
# importing library
import numpy as np
import datetime
import threading
import multiprocessing as mp
from matplotlib import pyplot as plt
from time import time
from scipy.stats import norm
%matplotlib inline
plt.style.use('bmh')

### Defineing diffusivity profile

In [2]:
import sympy
#sympy.init_printing()

z = sympy.symbols('z')

K0 = 1e-6# m * * 2 / s
K1 = 6e-3# m / s
Aalpha = 0.5
c=20
w=0

sym_Diffu =  K0 + K1 * z * sympy.exp(-Aalpha * z)
sym_dKdz = sympy.diff(sym_Diffu, z, 1)
sym_Beta = sympy.sqrt(2 * sym_Diffu)
sym_dBdz = sympy.diff(sym_Beta, z, 1)
sym_ddBdzz = sympy.diff(sym_Beta, z, 2)
sym_Alpha = w + sym_dKdz
sym_dAdz = sympy.diff(sym_Alpha, z, 1)
sym_ddAdzz = sympy.diff(sym_Alpha, z, 2)
sym_dABdz = sympy.diff(sym_Alpha * sym_Beta, z, 1)

Diffu  =  sympy.utilities.lambdify(z,          sym_Diffu,np)
dKdz   =  sympy.utilities.lambdify(z,          sym_dKdz,np)
Beta   =  sympy.utilities.lambdify(z,          sym_Beta,np)
dBdz   =  sympy.utilities.lambdify(z,          sym_dBdz,np)
ddBdzz=  sympy.utilities.lambdify(z,          sym_ddBdzz,np)
Alpha =  sympy.utilities.lambdify(z,      sym_Alpha,np)
dAdz  =  sympy.utilities.lambdify(z,      sym_dAdz,np)
ddAdzz=  sympy.utilities.lambdify(z,      sym_ddAdzz,np)
dABdz =  sympy.utilities.lambdify(z, sym_Alpha*sym_Beta,np)


In [3]:
#######
#Euler#
#######
def step_e(z,H,dt,N_sample):
    
    dW=np.random.normal(0,np.sqrt(dt),N_sample)
    
    a=dKdz(z)
    b=np.sqrt(2*Diffu(z))
    
    temp=z+a*dt+b*dW
    temp=np.where(temp<0, -temp ,temp)
    temp=np.where(temp>H, 2*H-temp,temp)
    return temp

def step_e_const(z,H,dt,N_sample):
    
    K0=3e-3
    dW=np.random.normal(0,np.sqrt(dt),N_sample)
    temp= z +np.sqrt(2*K0)*dW
    temp=np.where(temp<0, -temp ,temp)
    temp=np.where(temp>H, 2*H-temp,temp)
    return temp
########
#Visser#
########
def step_v(z,H,dt,N_sample):
    
    #dW=np.random.uniform(-1,1,N_sample)
    #r=1/3
    dW=np.random.normal(0,np.sqrt(dt),N_sample)
    r=dt
    
    a=dKdz(z)
    G=Diffu(z+a*dt/2)
    
    temp= z + a*dt + np.sqrt(2/r*dt*G)*dW
    temp=np.where(temp<0, -temp ,temp)
    temp=np.where(temp>H, 2*H-temp,temp)
    return temp

def step_v_const(z,H,dt,N_sample):
    K0=3e-3
    
    #dW=np.random.uniform(-1,1,N_sample)
    #r=1/3
    
    dW=np.random.normal(0,np.sqrt(dt),N_sample)
    r=dt
    
    temp= z + np.sqrt(2/r*dt*K0)*dW
    temp=np.where(temp<0, -temp ,temp)
    temp=np.where(temp>H, 2*H-temp,temp)
    return temp

##############
#Milstein 1nd#
##############
def step_m(z,H,dt,N_sample):
    dW=np.random.normal(0,np.sqrt(dt),N_sample)
    temp= z + (1/2)*dKdz(z)*(dW*dW+dt) + Beta(z)*dW
    temp=np.where(temp<0, -temp ,temp)
    temp=np.where(temp>H, 2*H-temp,temp)
    return temp

##############
#Milstein 2nd#
##############
def step_m2(z,H,dt,N_sample):
    dW=np.random.normal(0,np.sqrt(dt),N_sample)
    
    k=Diffu(z)
    dkdz=dKdz(z)
    ddkdz=dAdz(z)
    dddkdz=ddAdzz(z)
    sqrt2k=np.sqrt(2*k)
    
    a=dkdz
    da=ddkdz
    dda=dddkdz
    b= sqrt2k 
    db=dkdz/b
    ddb=ddkdz/b - ((dkdz)**2)/b**3
    ab=da*b+a*db
    
    temp= z + a*dt+b*dW+1/2*b*db*(dW*dW-dt)+1/2*(ab+1/2*ddb*b**2)*dW*dt+\
            1/2*(a*da+1/2*dda*b**2)*dt**2
    
    temp=np.where(temp<0, -temp ,temp)
    temp=np.where(temp>H, 2*H-temp,temp)
    return temp


### Defining functions for time adaptive

In [4]:
def oneStep(upperBound, lowerBound, TimeAdaptive, scheme, n, H, dt):

    maskBody = ((upperBound < TimeAdaptive) & (lowerBound > TimeAdaptive))

    zBody = TimeAdaptive[maskBody]
    zBody = scheme(zBody, H, dt, zBody.size)

    zEdge = TimeAdaptive[~maskBody]
    for j in range(n):
        zEdge = scheme(zEdge, H, dt / n, zEdge.size)

    TimeAdaptive[maskBody] = zBody
    TimeAdaptive[~maskBody] = zEdge

    return TimeAdaptive

def parallelBodyEdge(Tmax, dt, H, prosent, n, Testdepth, Np, Nbins, quene,
                     scheme):
    np.random.seed()
    upperBound = H * prosent
    lowerBound = H - upperBound
    Ntime = int(Tmax / dt)

    hist = np.zeros((Nbins - 1, ), 'i')
    TimeAdaptive = np.random.uniform(0, H, int(Np))

    for i in range(Ntime):

        TimeAdaptive = oneStep(upperBound, lowerBound, TimeAdaptive, scheme, n,
                               H, dt)
        temp0, _ = np.histogram(TimeAdaptive,
                                bins=np.linspace(0, Testdepth, Nbins))
        hist = hist + temp0

        if (i % int(Ntime / 100) == 0):
            print("\r %6.2f" % (i * 100 / Ntime + 1),
                  "%",
                  end="\r",
                  flush=True)

    quene.put(hist)
    return None

def RunSimulation(NumberOfThread, Tmax, dt, H, prosent, n, Testdepth, Np,
                  Nbins, scheme):
    print("Total number of process: ", NumberOfThread)
    SubNp = np.full((NumberOfThread, ), int(Np / NumberOfThread))
    SubNp[-1] = SubNp[-1] + (Np % NumberOfThread)

    Ntime = int(Tmax / dt)  #Number of time interval

    if (__name__ == '__main__'):
        threads = []
        quene = mp.Queue()

        for i in range(NumberOfThread):
            thread = mp.Process(target=parallelBodyEdge,
                                args=(Tmax, dt, H, prosent, n, Testdepth,
                                      SubNp[i], Nbins, quene, scheme))
            threads.append(thread)
            thread.start()  #starting calculation.

        for thread in threads:
            thread.join()  #waiting these processes finish.

    hist = np.zeros((Nbins - 1, ), 'i')

    for i in range(NumberOfThread):
        hist += quene.get()
        
    bins = np.linspace(0, Testdepth, Nbins)
    dz = bins[1] - bins[0]
    hist = (hist * H) / (Np * dz * (Ntime + 1))

    return hist

In [17]:
# Criterion checking

z=np.linspace(0, 10,1000)
profile=dAdz(z)
Maxdt=(1/np.max(np.abs(profile)))
print(Maxdt)

166.66666666666666


### E1

In [40]:
Np = 80000
Nbins = 600
Tmax = 4 * 3600  #Maximum time
dt = 1  #Delta time
H = 2
prosent = 0.1
n = 10
Testdepth = 0.1
NumberOfThread = 4

E1 = RunSimulation(NumberOfThread, Tmax, dt, H, prosent, n, Testdepth, Np,
                   Nbins, step_e)
E1_Const = RunSimulation(NumberOfThread, Tmax, dt, H, prosent, n, Testdepth,
                         Np, Nbins, step_e_const)
n = 1
E1_normal = RunSimulation(NumberOfThread, Tmax, dt, H, prosent, n, Testdepth,
                          Np, Nbins, step_e)

Total number of process:  4
Total number of process:  4
Total number of process:  4


In [41]:
bins = np.linspace(0, Testdepth, Nbins)
midpoints = bins[:-1] + (bins[1] - bins[0]) / 2

fig, ax = plt.subplots(ncols=1, figsize=(10, 10), sharey=True)
ax.plot(E1_normal, midpoints, linewidth=1.5, label="Non-time adaptive")
ax.plot(E1, midpoints, linewidth=1.5, label="Time adaptive $K_b(z)$")
ax.plot(E1_Const, midpoints, linewidth=1.5, label="Time adaptive constant $K$")
ax.set_xlabel('Concetration')
ax.set_ylabel('Depth /m')
ax.set_title('E1')
ax.invert_yaxis()
ax.set_xlim(0.6, 1.4)
ax.legend(loc = 4)
plt.tight_layout()